#Install **request** package on your LINUX machine

In [ ]:
!pip install requests

# Use Web API in python


Import the **request** package to send the request and get response and import **json** package to parse the response

In [ ]:
import requests
import json

Save our target URL in a variable for futural use

In [ ]:
url="http://api.openweathermap.org/data/2.5/weather?lat=34.0635363&lon=-118.4455592&units=Imperial&appid=a3276b42c9fbb481824cd2e6c23dd953"

Send out URL accessing request and get the return data which stored in ***response***

In [ ]:
# Query weather data using the api
response = requests.get(url)
# Read the json response into a Python dictionary
response_dict = json.loads(response.text)
print(response_dict)

{'coord': {'lon': -118.4456, 'lat': 34.0635}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 73.15, 'feels_like': 73.98, 'temp_min': 65.05, 'temp_max': 89.87, 'pressure': 998, 'humidity': 81}, 'visibility': 10000, 'wind': {'speed': 1.99, 'deg': 211, 'gust': 3}, 'clouds': {'all': 75}, 'dt': 1626191720, 'sys': {'type': 2, 'id': 2002107, 'country': 'US', 'sunrise': 1626180737, 'sunset': 1626231987}, 'timezone': -25200, 'id': 5408522, 'name': 'Westwood, Los Angeles', 'cod': 200}


Querying weather data of an arbitrary location

In [ ]:
# Specify coordinates
lat = 36
lon = -120
# Build the url with the specified coordinates
url = "http://api.openweathermap.org/data/2.5/weather?lat={0}&lon={1}&units=Imperial&appid=a3276b42c9fbb481824cd2e6c23dd953".format(lat, lon)
response = requests.get(url)
response_dict = json.loads(response.text)
# Read specific information in the response
print(response_dict['main'])
print(response_dict['main']['temp'])

{'temp': 79.52, 'feels_like': 79.52, 'temp_min': 79.52, 'temp_max': 81.81, 'pressure': 1010, 'humidity': 22, 'sea_level': 1010, 'grnd_level': 989}
79.52


# Building a simple Web API in python

## Install Flask

In [2]:
!pip install flask
!pip install flask-ngrok
!mkdir -p ~/projects/api
!cd ~/projects/api

## Simple web application that prints out provided lattitude and longitude.

In [ ]:
import flask
import requests
import json
from flask import request
# Enables flask to run on Colab
from flask_ngrok import run_with_ngrok

app = flask.Flask(__name__)
run_with_ngrok(app)

def get_weather_data(lat, lon):
  url = "http://api.openweathermap.org/data/2.5/weather?lat={0}&lon={1}&units=Imperial&appid=a3276b42c9fbb481824cd2e6c23dd953".format(lat, lon)
  response = requests.get(url)
  response_dict = json.loads(response.text)
  return response_dict

def get_weather_data_name(name):
  # Retrieve coordinates from name
  url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={0}&inputtype=textquery&fields=photos,formatted_address,name,rating,opening_hours,geometry&key=%20AIzaSyD8wnikJHxQUBzztfsD8RvkV_wJAgi1-d8".format(name)
  response_places = requests.get(url)
  response_places_dict = json.loads(response_places.text)
  location = response_places_dict["candidates"][0]["geometry"]["location"]
  lat = location["lat"]
  lon = location["lng"]
  # Retrieve weather from coordinates
  return get_weather_data(lat,lon)

# Route to home screen
@app.route('/', methods=['GET'])
def home():
  return "<h1>LACC </h1><p>This site is a prototype API for something.</p>"

def shutdown_server():
  func = flask.request.environ.get('werkzeug.server.shutdown')
  if func is None:
    raise RuntimeError('Not running with the server')
  func()

# Shutdown the server
@app.route('/shutdown', methods=['GET'])
def shutdown():
  shutdown_server()
  return 'Server shutting down...'

# Query
@app.route('/weather', methods=['GET'])
def api_weather():
  '''
  Check if coordinate data is provided as part of the url
  '''
  # Coordinate data
  if ('lat' in request.args) and ('lon' in request.args):
    lat = int(request.args['lat'])
    lon = int(request.args['lon'])
    response_dict = get_weather_data(lat,lon)
    response_return = "Weather at lat {lat}, lon {lon} is {weather}. Temperature is {temp}°F.".format(lat=lat, lon=lon, weather=response_dict['weather'][0]['main'], temp=response_dict['main']['temp'])
    return response_return
  # Location data
  elif ('location' in request.args):
    coordinates = request.args['location'].split(',')
    lat = coordinates[0].rstrip()
    lon = coordinates[1].rstrip()
    response_dict = get_weather_data(lat,lon)
    response_return = "Weather at location {loc} is {weather}. Temperature is {temp}°F.".format(loc=request.args['location'], weather=response_dict['weather'][0]['main'], temp=response_dict['main']['temp'])
    return response_return
  # Location name data
  elif ('name' in request.args):
    response_dict = get_weather_data_name(request.args['name'])
    response_return = "Weather at {name} is {weather}. Temperature is {temp}°F.".format(name=request.args['name'], weather=response_dict['weather'][0]['main'], temp=response_dict['main']['temp'])
    return response_return
  else:
    return 'Unrecognized location format'


app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://3f45f7bf2064.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [13/Jul/2021 16:40:36] "GET /weather?name=los%20angeles HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 16:40:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [13/Jul/2021 16:45:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 16:45:16] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [13/Jul/2021 16:47:36] "GET /weather?name=Shanghai HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 16:47:45] "GET /weather?name=Los%20Vegas HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 16:47:50] "GET /weather?name=Somewhere HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 16:51:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 16:55:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:11:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:31:31] "GET / HTTP/1.1" 200 -
